In [1]:
import sys
sys.path.append('/home/shiftpub/Dynamic_AMM/inf_step_exp')
from vector_train import AMM
import numpy as np

distribute_amm = AMM(
    L=1000000,
    gamma=0.003,
    sigma=0.5,
    delta_t=1,
    mu=0.0,
    fee_model='distribute',
    fee_source='incoming',
    device='cuda'
)

states = distribute_amm.generate_training_data(num_samples=100)
print(f"states shape: {states.shape}")
print(states[:4])

states shape: torch.Size([100, 3])
tensor([[1.1080e+00, 9.5000e+05, 1.0526e+06],
        [1.1057e+00, 9.5101e+05, 1.0515e+06],
        [1.1033e+00, 9.5202e+05, 1.0504e+06],
        [1.1010e+00, 9.5303e+05, 1.0493e+06]], device='cuda:0')


In [2]:
states_np = states.cpu().numpy()
p = states_np[:, 0]
x = states_np[:, 1]
y = states_np[:, 2]
print(f"states_np shape: {states_np.shape}")
print(f"p shape: {p.shape}")
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")
print(states_np[:4])

states_np shape: (100, 3)
p shape: (100,)
x shape: (100,)
y shape: (100,)
[[1.10803324e+00 9.50000000e+05 1.05263158e+06]
 [1.10568073e+00 9.51010101e+05 1.05151354e+06]
 [1.10333570e+00 9.52020202e+05 1.05039788e+06]
 [1.10099813e+00 9.53030303e+05 1.04928458e+06]]


In [3]:
mu = 0.0
sigma = 0.5
delta_t = 1

log_p = np.log(p)
print(f"log_p shape: {log_p.shape}")
log_p_mean = log_p[:, np.newaxis] + (mu - 0.5 * sigma**2) * delta_t
log_p_std = sigma * np.sqrt(delta_t)
print(f"log_p_mean shape: {log_p_mean.shape}")
print(f"log_p_std shape: {log_p_std.shape}")
# Pre-compute quadrature points and weights once
num_points = 500
points, weights = np.polynomial.legendre.leggauss(num_points)
print(f"points shape: {points.shape}")
# points = points[np.newaxis, :]
# print(f"points shape: {points.shape}")
log_p_min = log_p_mean - 3 * log_p_std
log_p_max = log_p_mean + 3 * log_p_std
# log_p_points = 0.5 * (log_p_max - log_p_min) * points[np.newaxis, :] + 0.5 * (log_p_max + log_p_min)
log_p_points = 0.5 * (log_p_max - log_p_min) * points + 0.5 * (log_p_max + log_p_min)

print(log_p_points.shape)

log_p shape: (100,)
log_p_mean shape: (100, 1)
log_p_std shape: ()
points shape: (500,)
(100, 500)


In [4]:
gamma = 0.003
transformed_weights = weights[np.newaxis, :] * 0.5 * (log_p_max - log_p_min)
print(f"transformed_weights shape: {transformed_weights.shape}")
price_ratio = y / x  # (batch_size,)
p_upper = price_ratio / (1 - gamma)  # (batch_size,)
p_lower = price_ratio * (1 - gamma)  # (batch_size,)

print(f"price_ratio shape: {price_ratio.shape}")
print(f"p_upper shape: {p_upper.shape}")
print(f"p_lower shape: {p_lower.shape}")



transformed_weights shape: (100, 500)
price_ratio shape: (100,)
p_upper shape: (100,)
p_lower shape: (100,)


In [5]:
p_points = np.exp(log_p_points)
print(f"p_points shape: {p_points.shape}")

p_upper = p_upper[:, np.newaxis]
p_lower = p_lower[:, np.newaxis]

print(f"p_upper shape: {p_upper.shape}")
print(f"p_lower shape: {p_lower.shape}")

up_mask = p_points > p_upper
down_mask = p_points < p_lower
within_mask = ~(up_mask | down_mask)

print(up_mask.shape)
print(down_mask.shape)
print(within_mask.shape)





p_points shape: (100, 500)
p_upper shape: (100, 1)
p_lower shape: (100, 1)
(100, 500)
(100, 500)
(100, 500)


In [7]:
print(f"original x shape: {x.shape}")
print(f"expanded x shape: {x[:, np.newaxis].shape}")
print(f"repeated x shape: {x[:, np.newaxis].repeat(500, axis=1).shape}")

original x shape: (100,)
expanded x shape: (100, 1)
repeated x shape: (100, 500)


In [23]:
a = np.array([1, 2, 3, 4, 5, 6])
a = a[np.newaxis,:].repeat(3, axis=0)
print(a.shape)
print(a)

b = np.array([10, 20, 30])[:, np.newaxis]
print(b.shape)
print(b)
print(a - b)





(3, 6)
[[1 2 3 4 5 6]
 [1 2 3 4 5 6]
 [1 2 3 4 5 6]]
(3, 1)
[[10]
 [20]
 [30]]
[[ -9  -8  -7  -6  -5  -4]
 [-19 -18 -17 -16 -15 -14]
 [-29 -28 -27 -26 -25 -24]]
